# Prep des macro-domaines DUFLT

Version février 2026 intégrant les 40 domaines de Denis

In [1]:
import pandas as pd
import os

import datetime

# date as "yyyymmdd"
suffix = datetime.datetime.now().strftime("%Y%m%d")
suffix

'20260209'

In [2]:
df = pd.read_csv(os.path.join('..', 'data', '1743_LeRobert', 'Trevoux1743.tsv'), sep='\t', encoding='utf-8')
df.head()

,book,volume,numero,head,id,type,src-domain,text
0,DUFLT_1743,1,1,A,1,mainEntry,NaN,A est la première Lettre de l'Alphabet Françoi...
1,DUFLT_1743,1,2,AAHUS,1,mainEntry,NaN,AAHUS Aahusium. Ville de l'Evéché de Munster. ...
2,DUFLT_1743,1,3,AAR,1,mainEntry,NaN,"AAR ou AHR. subst. Aara, Abrinca. Rivière d'Al..."
3,DUFLT_1743,1,3,AAR,1,mainEntry,NaN,"AAR Arula ou Arola, & non pas Arosa, comme on ..."
4,DUFLT_1743,1,4,AARBRER,1,mainEntry,NaN,AARBRER Terme ancien qui n'est pas aujourd'hui...


In [3]:
df_domaines_denis = pd.read_excel('Conversion_DUFLT_protodesignants_MacroDomaines_Denis.xlsx')
df_domaines_denis.rename(columns={"subordinate_domain":"src-domain", "Macro-domaines correspondants": "macro-domain"}, inplace=True)
df_domaines_denis = df_domaines_denis[["src-domain", "macro-domain"]]

df_domaines_denis['macro-domain'] = df_domaines_denis['macro-domain'].replace('histoire', 'Histoire')
df_domaines_denis['macro-domain'] = df_domaines_denis['macro-domain'].replace(' Religion', 'Religion')
df_domaines_denis['macro-domain'] = df_domaines_denis['macro-domain'].replace('Métiers [ métal, minéral,  dérivés]', 'Métiers [métal, minéral, dérivés]')
df_domaines_denis['macro-domain'] = df_domaines_denis['macro-domain'].replace('Agriculture - Économie domestique', 'Agriculture - Économie rustique')

df_domaines_denis.head()

/Users/lmoncla/git/github.com/lmoncla/trevoux/.venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,src-domain,macro-domain
0,Terme d'Agriculture,Agriculture - Économie rustique
1,Terme d'Agriculture & de Coutumes,Agriculture - Économie rustique
2,Terme d'Agriculture & de Jardinage,Agriculture - Économie rustique
3,Terme d'Agriculture & de labourage,Agriculture - Économie rustique
4,Terme d'Agriculture & de Vigneron,Agriculture - Économie rustique


In [ ]:
print(df_domaines_denis['macro-domain'].unique())
print(len(df_domaines_denis['macro-domain'].unique()))

In [4]:
# lowercase src-domain columns
df_domaines_denis['src-domain'] = df_domaines_denis['src-domain'].str.lower()
df['src-domain'] = df['src-domain'].str.lower()

In [5]:
df = pd.merge(df, df_domaines_denis, on='src-domain', how='left')
df.head()

,book,volume,numero,head,id,type,src-domain,text,macro-domain
0,DUFLT_1743,1,1,A,1,mainEntry,NaN,A est la première Lettre de l'Alphabet Françoi...,NaN
1,DUFLT_1743,1,2,AAHUS,1,mainEntry,NaN,AAHUS Aahusium. Ville de l'Evéché de Munster. ...,NaN
2,DUFLT_1743,1,3,AAR,1,mainEntry,NaN,"AAR ou AHR. subst. Aara, Abrinca. Rivière d'Al...",NaN
3,DUFLT_1743,1,3,AAR,1,mainEntry,NaN,"AAR Arula ou Arola, & non pas Arosa, comme on ...",NaN
4,DUFLT_1743,1,4,AARBRER,1,mainEntry,NaN,AARBRER Terme ancien qui n'est pas aujourd'hui...,NaN


In [6]:
df['src-domain'].notna().sum()

np.int64(13476)

In [7]:
# macro_domain not NaN
df[df['macro-domain'].notna()]

,book,volume,numero,head,id,type,src-domain,text,macro-domain
15,DUFLT_1743,1,13,ABADIR,1,mainEntry,terme de mythologie,"ABADIR ou ABADDIR ; car Priscien, qui nous a c...",Antiquité
16,DUFLT_1743,1,14,ABAEUZ,1,mainEntry,terme de coutume,ABAEUZ Terme de Coutume . Biens abaeuz. Bona v...,Droit Jurisprudence
17,DUFLT_1743,1,15,ABAJOUR,1,mainEntry,terme d'architecture,"ABAJOUR Terme d'Architecture , Spiraculum, esp...",Architecture
18,DUFLT_1743,1,15,ABAJOUR,2,relatedEntry,terme de botanique,ABAJOUR Abajour . s. m. Terme de Botanique . S...,Histoire naturelle
25,DUFLT_1743,1,19,ABAISSER,3,relatedEntry,termes de fauconnerie,"ABAISSER Abaisser , signifie aussi en morale, ...",Chasse-pêche
...,...,...,...,...,...,...,...,...,...
95100,DUFLT_1743,6,5424,ZUNDANASTAN,1,mainEntry,terme de relation,ZUNDANASTAN Terme de Relation . C'est le livre...,Voyage
95101,DUFLT_1743,6,5424,ZUNDANASTAN,1,mainEntry,terme de relation,ZUNDANASTAN Terme de Relation . C'est le livre...,Voyage
95103,DUFLT_1743,6,5426,ZUNITTER,1,mainEntry,terme de philosophie hermétique,ZUNITTER ou ZITTER. s. m. Terme de Philosophie...,Philosophie
95105,DUFLT_1743,6,5428,ZURANDALE,1,mainEntry,terme de fleuriste,ZURANDALE Terme de Fleuriste . Nom de tulipe. ...,Métiers [autres]


In [9]:
df[df['src-domain'].notna() & ~df['macro-domain'].notna()]

,book,volume,numero,head,id,type,src-domain,text,macro-domain
20,DUFLT_1743,1,17,ABAISSE,1,mainEntry,terme de pâtissier,ABAISSE terme de Pâtissier . C'est la pâte qui...,NaN
116,DUFLT_1743,1,76,ABDOMEN,1,mainEntry,terme de médecine,"ABDOMEN Terme de Médecine , qui signifie la pa...",NaN
149,DUFLT_1743,1,103,ABIGEAT,1,mainEntry,terme de droit romain,ABIGEAT Terme de Droit Romain . L'Abigeat est ...,NaN
176,DUFLT_1743,1,124,ABOKELLE,1,mainEntry,terme de négotiant en égypte & de relation,ABOKELLE Terme de Négotiant en Égypte & de Rel...,NaN
181,DUFLT_1743,1,127,ABOLITION,1,mainEntry,terme de chancellerie,ABOLITION Terme de Chancellerie . Abolitio cri...,NaN
...,...,...,...,...,...,...,...,...,...
94282,DUFLT_1743,6,4674,VULNÉRAIRE,1,mainEntry,terme de médecine,VULNÉRAIRE Terme de Médecine . Épithète qu'on ...,NaN
94873,DUFLT_1743,6,5219,ZAIN,1,mainEntry,terme de manège,"ZAIN Terme de Manège , se dit d'un cheval qui ...",NaN
94911,DUFLT_1743,6,5249,ZARCA,1,mainEntry,termes de philosophie hermétique,"ZARCA en termes de Philosophie hermétique , c'...",NaN
94988,DUFLT_1743,6,5319,ZERCI,1,mainEntry,termes de philosophie hermétique,"ZERCI En termes de Philosophie hermétique , c'...",NaN


In [10]:
df.to_csv("Trevoux1743_macro_"+suffix+".tsv", sep="\t", index=False)
df.to_excel("Trevoux1743_macro_"+suffix+".xlsx", index=False)